In [ ]:
from agents import Agent, Runner, trace, Tool, SQLiteSession
from agents.run_context import RunContextWrapper
from agents.mcp import MCPServerStdio, create_static_tool_filter
from IPython.display import Markdown, display
from dotenv import load_dotenv
load_dotenv()

In [ ]:
### Using Third-Party MCP Server 
# async with MCPServerStdio(
#     params={
#         "command": "npx",
#         "args": ["-y", "@modelcontextprotocol/server-filesystem", "./"],
#     },
#     client_session_timeout_seconds=30
# ) as server:
#     with trace("mcp"):
#         agent = Agent(name="Test", instructions="You are a helpful assistant", model="gpt-4o-mini", mcp_servers=[server])
#         result = await Runner.run(agent, "Create a new file test.txt with content 'Hello world'")
#         print(result.final_output)

In [ ]:
## Both Custom and Third-Party Servers Configuration
params = [
    {
        "name": "Pdf master - who can extract text from pdf",
        "command": "uv",
        "args": ["run", "./mcp_servers/pdf-master.py"],
    },
    {
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-filesystem", "./files"],
    }
]

servers = [MCPServerStdio(params=param, client_session_timeout_seconds=30) for param in params]

In [ ]:
# Connecting servers 
for server in servers:
    await server.connect()

# Using MCP Server with Agent
with trace("non_filter_mcp"):
    agent = Agent(name="mympc", instructions="You are a helpful assistant", model="gpt-4o-mini", mcp_servers=servers)
    result = await Runner.run(agent, "extract this pdf './files/example.pdf' and make summary and important notes from extracted text and save it to data.md")
    display(Markdown(result.final_output))

In [ ]:
server1 = MCPServerStdio(
    params={
        "name": "Pdf master - who can extract text from pdf",
        "command": "uv",
        "args": ["run", "./mcp_servers/pdf-master.py"],
    },
    client_session_timeout_seconds=30
)
server2 = MCPServerStdio(
    params={
        "name":"File System Server",
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-filesystem", "./files"],
    },
    tool_filter=create_static_tool_filter(
        allowed_tool_names=["read_file", "write_file", "create_directory"],
        # blocked_tool_names=["move_file"]
    ),
    client_session_timeout_seconds=30
)

### Check tools of server2 
await server2.connect()
run_context = RunContextWrapper(context=None)
agent = agent = Agent(name="Test", instructions="You are a helpful assistant", model="gpt-4o-mini")
tools = await server2.list_tools(run_context, agent)
print(len(tools))
for tool in tools:
    print(tool.name)


In [ ]:
# Connecting servers 
for server in [server1, server2]:
    await server.connect()

# Using MCP Server with Agent
with trace("filtered_mcp"):
    agent = Agent(name="mympc", instructions="You are a helpful assistant", model="gpt-4o-mini", mcp_servers=servers)
    result = await Runner.run(agent, "extract this pdf './files/example.pdf' and make summary and important notes from extracted text and save it to data.md")
    display(Markdown(result.final_output))